# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [33]:
! uv add seaborn
! uv add opencc
! uv add --upgrade scikit-learn
! uv add ipykernel
! uv add torch torchvision
! uv add numpy pandas matplotlib tqdm

Resolved 79 packages in 15ms
Audited 73 packages in 11ms
Resolved 79 packages in 11ms
Audited 73 packages in 0.87ms
Resolved 79 packages in 404ms                                        
Audited 73 packages in 2ms
Resolved 79 packages in 11ms
Audited 73 packages in 0.93ms
Resolved 79 packages in 12ms
Audited 73 packages in 0.88ms
Resolved 79 packages in 11ms
Audited 73 packages in 0.87ms


In [57]:
!uv pip freeze > requirements.txt

In [58]:
!cat requirements.txt

asttokens==3.0.0
comm==0.2.3
contourpy==1.3.3
cycler==0.12.1
debugpy==1.8.17
decorator==5.2.1
executing==2.2.1
filelock==3.19.1
fonttools==4.60.1
fsspec==2025.9.0
ipykernel==6.30.1
ipython==9.6.0
ipython-pygments-lexers==1.1.1
jedi==0.19.2
jinja2==3.1.6
joblib==1.5.2
jupyter-client==8.6.3
jupyter-core==5.8.1
kiwisolver==1.4.9
markupsafe==3.0.3
matplotlib==3.10.6
matplotlib-inline==0.1.7
mpmath==1.3.0
nest-asyncio==1.6.0
networkx==3.5
numpy==2.3.3
nvidia-cublas-cu12==12.8.4.1
nvidia-cuda-cupti-cu12==12.8.90
nvidia-cuda-nvrtc-cu12==12.8.93
nvidia-cuda-runtime-cu12==12.8.90
nvidia-cudnn-cu12==9.10.2.21
nvidia-cufft-cu12==11.3.3.83
nvidia-cufile-cu12==1.13.1.3
nvidia-curand-cu12==10.3.9.90
nvidia-cusolver-cu12==11.7.3.90
nvidia-cusparse-cu12==12.5.8.93
nvidia-cusparselt-cu12==0.7.1
nvidia-nccl-cu12==2.27.3
nvidia-nvjitlink-cu12==12.8.93
nvidia-nvtx-cu12==12.8.90
opencc==1.1.9
packaging==25.0
pandas==2.3.3
parso==0.8.5
pexpect==4.9.0
pillow==11.3.0
platformdirs==4.4.0
prompt-toolkit==3.0.52

In [34]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

In [35]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,src,tgt
0,14*(43+20)=,882
1,(6+1)*5=,35
2,13+32+29=,74
3,31*(3-11)=,-248
4,24*49+1=,1177


In [ ]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])      # src = src + tgt
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [ ]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every character in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to convert characters to ids, while id_to_char is the opposite

def tokenize_arithmetic(text):
    """
    將算術表達式按字符級別分詞
    例如: "14*(43+20)=882" -> ['1','4','*','(','4','3','+','2','0',')','=','8','8','2']
    """
    return list(text)

# 首先添加特殊字符
char_to_id['<pad>'] = 0
char_to_id['<eos>'] = 1
id_to_char[0] = '<pad>'
id_to_char[1] = '<eos>'

# 收集所有獨特字符
unique_chars = set()

# 只從 train dataset 收集字符
# 每次取出一行算式
for text in df_train['src']:
    unique_chars.update(text)

# 為每個字符分配 ID (從 2 開始，因為 0 和 1 已經被特殊字符使用)
char_id = 2

# 將獨特字符按排序順序添加到字典中，用於後續將字符轉換為 token id 與反轉換。
for char in sorted(unique_chars):  # 排序以確保一致性
    if char not in char_to_id:
        char_to_id[char] = char_id
        id_to_char[char_id] = char
        char_id += 1

vocab_size = len(char_to_id)
print('Vocab size: {}'.format(vocab_size))
print('Characters in vocabulary:', sorted(char_to_id.keys()))

Vocab size: 18
Characters in vocabulary: ['(', ')', '*', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '<eos>', '<pad>', '=']


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [ ]:
# Write your code here

from tqdm import tqdm

def process_data(df):
    """
    將算式轉換為 ID 序列
    輸入: "14+20=34"
    
    關鍵：使用自回歸訓練方式
    - 模型輸入: [1,4,+,2,0,=,3,4] (不包含<eos>)
    - 模型標籤: [4,+,2,0,=,3,4,<eos>] (向右偏移一位)
    - 在等號之前的位置，標籤用 <pad> 遮蔽（計算損失時忽略）
    
    這樣訓練時，模型會學習：
    - 看到 "1+1=" 後，預測 "2"
    - 看到 "1+1=2" 後，預測 "<eos>"
    """
    print(f"Processing {len(df)} samples...")
    
    char_id_lists = []
    label_id_lists = []
    
    for text in tqdm(df['src']): # 每次取出一行算式
        # 找到等號位置
        equal_idx = text.find('=')
        
        if equal_idx == -1:
            continue  # 跳過沒有等號的樣本 (照理來講不應該出現)
            
        # 轉換為 ID（不包含 <eos>）
        char_ids = [char_to_id[c] for c in text] # 從算式取出每個字符並轉換為對應的 id
        
        # 創建標籤：向右偏移一位，最後加上 <eos>
        label_ids = char_ids[1:] + [char_to_id['<eos>']]
        
        # 遮蔽等號之前的標籤（用 <pad>，損失函數會忽略）
        label_ids = [char_to_id['<pad>']] * equal_idx + label_ids[equal_idx:]
        
        # 輸入不包含 <eos>
        char_id_lists.append(char_ids)
        label_id_lists.append(label_ids)
    
    # 創建結果DataFrame
    result_df = pd.DataFrame({
        'src': df['src'].iloc[:len(char_id_lists)].values,
        'char_id_list': char_id_lists,
        'label_id_list': label_id_lists
    })
    
    return result_df

# 處理訓練和評估數據
import time

print("=== 數據處理開始 ===")
start_time = time.time()

df_train_processed = process_data(df_train)
df_eval_processed = process_data(df_eval)

end_time = time.time()
print(f"數據處理完成，耗時: {end_time - start_time:.2f}秒")

# 顯示處理後的數據
print("\n=== 處理結果驗證（新的自回歸方式）===")
sample_idx = 0
sample_src = df_train_processed.iloc[sample_idx]['src']
sample_input = df_train_processed.iloc[sample_idx]['char_id_list']
sample_label = df_train_processed.iloc[sample_idx]['label_id_list']

print(f"原始數據範例: {sample_src}")
print(f"模型輸入字符: {[id_to_char[id] for id in sample_input]}")
print(f"模型標籤字符: {[id_to_char[id] for id in sample_label]}")

# 找到等號位置來驗證
equal_pos = sample_src.find('=')

print(f"\n等號位置: {equal_pos}")
print(f"問題部分輸入: {[id_to_char[id] for id in sample_input[:equal_pos+1]]}")
print(f"答案部分輸入: {[id_to_char[id] for id in sample_input[equal_pos+1:]]}")
print(f"問題部分標籤（應該都是<pad>）: {[id_to_char[id] for id in sample_label[:equal_pos]]}")
print(f"等號位置標籤（應該是答案第一個字符）: {id_to_char[sample_label[equal_pos]]}")
print(f"答案部分標籤: {[id_to_char[id] for id in sample_label[equal_pos+1:]]}")

print("\n解釋：")
print(f"當模型看到 '{sample_src[:equal_pos+1]}'，它應該預測 '{id_to_char[sample_label[equal_pos]]}'")
print(f"當模型看到 '{sample_src[:equal_pos+2]}'，它應該預測 '{id_to_char[sample_label[equal_pos+1]]}'")

df_train_processed.head()


=== 數據處理開始 ===
Processing 2369250 samples...


100%|██████████| 2369250/2369250 [00:02<00:00, 803557.84it/s]


Processing 263250 samples...


100%|██████████| 263250/263250 [00:00<00:00, 809087.61it/s]


數據處理完成，耗時: 4.45秒

=== 處理結果驗證（新的自回歸方式）===
原始數據範例: 14*(43+20)=882
模型輸入字符: ['1', '4', '*', '(', '4', '3', '+', '2', '0', ')', '=', '8', '8', '2']
模型標籤字符: ['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '8', '8', '2', '<eos>']

等號位置: 10
問題部分輸入: ['1', '4', '*', '(', '4', '3', '+', '2', '0', ')', '=']
答案部分輸入: ['8', '8', '2']
問題部分標籤（應該都是<pad>）: ['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
等號位置標籤（應該是答案第一個字符）: 8
答案部分標籤: ['8', '2', '<eos>']

解釋：
當模型看到 '14*(43+20)='，它應該預測 '8'
當模型看到 '14*(43+20)=8'，它應該預測 '8'


,src,char_id_list,label_id_list
0,14*(43+20)=882,"[8, 11, 4, 2, 11, 10, 5, 9, 7, 3, 17, 15, 15, 9]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 9, 1]"
1,(6+1)*5=35,"[2, 13, 5, 8, 3, 4, 12, 17, 10, 12]","[0, 0, 0, 0, 0, 0, 0, 10, 12, 1]"
2,13+32+29=74,"[8, 10, 5, 10, 9, 5, 9, 16, 17, 14, 11]","[0, 0, 0, 0, 0, 0, 0, 0, 14, 11, 1]"
3,31*(3-11)=-248,"[10, 8, 4, 2, 10, 6, 8, 8, 3, 17, 6, 9, 11, 15]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 9, 11, 15, 1]"
4,24*49+1=1177,"[9, 11, 4, 11, 16, 5, 8, 17, 8, 8, 14, 14]","[0, 0, 0, 0, 0, 0, 0, 8, 8, 14, 14, 1]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [39]:
batch_size = 1024*3
epochs = 30
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

torch.cuda.empty_cache()

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)  # Write your code here
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences.iloc[index]['char_id_list']  # Write your code here
        y = self.sequences.iloc[index]['label_id_list']  # Write your code here
        return x, y

# collate function, used to build dataloader
# 接收一個 batch 的原始資料，將其統一格式，使其能夠輸入進模型(轉換為 tensor，並進行 padding 至相同長度)
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [ ]:
ds_train = Dataset(df_train_processed[['char_id_list', 'label_id_list']]) # 不需要 df_train_processed['src']
ds_eval = Dataset(df_eval_processed[['char_id_list', 'label_id_list']])

In [ ]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(ds_train,  # Write your code here
                                       batch_size=batch_size, 
                                       shuffle=True, 
                                       collate_fn=collate_fn,
                                       num_workers=8,
                                       pin_memory=True,  # 加速GPU傳輸
                                       prefetch_factor=4)  # 預取更多批次

dl_eval = torch.utils.data.DataLoader(ds_eval,  # Write your code here
                                      batch_size=batch_size*5, 
                                      shuffle=False, 
                                      collate_fn=collate_fn,
                                      num_workers=8,
                                      pin_memory=True,  # 加速GPU傳輸
                                      prefetch_factor=8)  # 預取更多批次

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CharLSTM(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharLSTM, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.lstm_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.lstm_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),          # 自己新增的
            torch.nn.LayerNorm(hidden_dim), # 自己新增的
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
    
    # 使用 teacher forwarding (標作作法僅訓練時使用) (但是實驗發現在本任務當中效果和使用 generator 差不多，但是快很多)
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        # 字符嵌入
        batch_x = self.embedding(batch_x)
        
        # 將輸入的 tensor 轉成特殊的 PackedSequence 格式，讓 LSTM 可以跳過 padding，提升速度。 (LSTM 以後需要用 pad_packed_sequence 轉回來。)
        # 與 collate_fn 當中輸入的參數不同，上面使用 pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x, batch_first=True, padding_value=char_to_id['<pad>'])
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,         # 算式未 padding 前的原始實際長度
                                                          batch_first=True,
                                                          enforce_sorted=False) # 不需要按照長度排序 batch
        
        # 通過兩層LSTM
        batch_x, _ = self.lstm_layer1(batch_x)
        batch_x, _ = self.lstm_layer2(batch_x)
        
        # 將 PackedSequence 格式轉回普通 tensor，padding 的位置會被填充為 0。
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x, batch_first=True)
        
        # 通過線性層得到每個位置的字符預測
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    # 最真實的 forward 相當於不使用 teacher forcing ， val 和 test 時的標準做法。
    def generator(self, start_char, max_len=20, temperature=1.0, use_sampling=False):
        """
        改進的生成器
        - temperature: 控制隨機性，越高越隨機
        - use_sampling: 是否使用採樣而不是貪婪解碼
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        # 檢查是否所有字符都在詞彙表中
        try:
            char_ids = [char_to_id[c] for c in start_char]
        except KeyError as e:
            raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 逐字符生成
        for _ in range(max_len - len(char_ids)): # 最大長度 - 算式已有長度
            x = torch.tensor([char_ids]).to(model_device)
            x_len = torch.tensor([len(char_ids)])
            
            with torch.no_grad():
                y = self.forward(x, x_len)
            
            # 取最後一個時間步的輸出 (batch_size, seq_len, vocab_size)
            logits = y[0, -1, :] / temperature # temperature 越小越接近貪婪(取最佳答案)，溫度越大越隨機。
            
            # 禁止生成 <pad>
            logits[char_to_id['<pad>']] = -float('inf')
            
            if use_sampling:
                # 使用溫度採樣
                probs = torch.softmax(logits, dim=-1)             # softmax 將 logits 轉為機率分佈
                next_char_id = torch.multinomial(probs, 1).item() # 按照機率隨機抽取結果
            else:
                # 貪婪解碼
                next_char_id = logits.argmax().item()
            
            if next_char_id == char_to_id['<eos>']:
                char_ids.append(next_char_id)
                break
            
            char_ids.append(next_char_id)
            
        # 將字符 IDs 轉換回字符串
        return ''.join([id_to_char[char_id] for char_id in char_ids])
    
    def batch_generator(self, start_sequences, max_len=20, temperature=1.0, use_sampling=False):
        """
        批次生成器 - 大幅提升評估速度！
        
        參數:
            start_sequences: list of strings，每個是問題部分（例如 ["1+1=", "5*6="]）
            max_len: 最大生成長度
            temperature: 控制隨機性
            use_sampling: 是否使用採樣
        
        返回:
            list of strings，每個是完整生成結果
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        batch_size = len(start_sequences)
        
        # 將每個起始序列轉換為 ID 列表，並檢查字符是否在詞彙表中
        batch_char_ids = []
        for seq in start_sequences:
            try:
                char_ids = [char_to_id[c] for c in seq]
                batch_char_ids.append(char_ids)
            except KeyError as e:
                raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 記錄每個序列是否已經完成（遇到 <eos> 或達到最大長度）
        finished = [False] * batch_size
        
        # 逐步生成，最多生成 max_len 個字符
        for step in range(max_len):
            # 如果所有序列都已完成，提前退出
            if all(finished):
                break
            
            # 準備當前批次的輸入
            current_batch = [torch.tensor(ids) for ids in batch_char_ids]
            batch_lens = torch.LongTensor([len(ids) for ids in batch_char_ids])
            
            # Padding 到相同長度
            padded_batch = torch.nn.utils.rnn.pad_sequence(
                current_batch, 
                batch_first=True, 
                padding_value=char_to_id['<pad>']
            ).to(model_device)
            
            # 批次前向傳播
            with torch.no_grad():
                batch_output = self.forward(padded_batch, batch_lens)
            
            # 對每個序列，取最後一個時間步的輸出
            for i in range(batch_size):
                if finished[i]:
                    continue
                
                # 獲取該序列最後一個位置的 logits
                last_pos = batch_lens[i] - 1
                logits = batch_output[i, last_pos, :] / temperature
                
                # 禁止生成 <pad>
                logits[char_to_id['<pad>']] = -float('inf')
                
                # T:按照prob取出token當作答案(隨機), F:取prob最高的token當作答案
                if use_sampling:
                    probs = torch.softmax(logits, dim=-1)
                    next_char_id = torch.multinomial(probs, 1).item()
                else:
                    next_char_id = logits.argmax().item()
                
                # 添加到序列
                batch_char_ids[i].append(next_char_id)
                
                # 檢查是否遇到 <eos>
                if next_char_id == char_to_id['<eos>']:
                    finished[i] = True
        
        # 將 ID 列表轉換回字符串
        results = [''.join([id_to_char[char_id] for char_id in ids]) 
                   for ids in batch_char_ids]
        
        return results


In [14]:
torch.manual_seed(2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Write your code here. Specify a device (cuda or cpu)
print(f"Using device: {device}")


model = CharLSTM(vocab_size,
                embed_dim,
                hidden_dim)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

model = torch.compile(model, dynamic=True)

Using device: cuda
Model parameters: 1,128,210


## 損失、優化器、排程器

In [15]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])  # Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)  # Write your code here. Use Adam or AdamW for Optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr * 0.1)

# 測試批次生成器效能

比較單個生成 vs 批次生成的速度差異

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [16]:
torch.cuda.empty_cache()

## 訓練LSTM (驗證使用generate (真正公平的驗證)(但很慢))

In [ ]:
from tqdm import tqdm
from copy import deepcopy
import time
from datetime import datetime, timedelta

model = model.to(device)
model.train()
i = 0

# 記錄最佳模型
best_em = 0
best_model_state = None

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()
print("=" * 60)
print(f"訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    # The process bar
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # ============================================
    # 評估：使用批次生成方式（高效且無teacher forcing）
    # ============================================
    model.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # 收集這個批次中所有的問題部分
            question_parts = []
            ground_truth_answers = []
            
            for idx in range(len(batch_x)):
                # 獲取輸入序列
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                # 找到等號位置
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                # 只給模型問題部分（包含等號），不給答案
                question_part = x_str[:equal_pos+1]  # 例如 "1+1="
                question_parts.append(question_part)
                
                # 保存真實答案
                ground_truth_answer = x_str[equal_pos+1:]
                ground_truth_answers.append(ground_truth_answer)
            
            # *** 關鍵優化：批次生成所有答案 ***
            if len(question_parts) > 0:
                generated_results = model.batch_generator(question_parts, max_len=20)
                
                # 評估每個生成結果
                for gen_result, gt_answer in zip(generated_results, ground_truth_answers):
                    # 提取生成的答案部分
                    gen_equal_pos = gen_result.find('=')
                    if '<eos>' in gen_result:
                        eos_pos = gen_result.find('<eos>')
                        pred_answer = gen_result[gen_equal_pos+1:eos_pos]
                    else:
                        pred_answer = gen_result[gen_equal_pos+1:]
                    
                    # 比較答案（完全匹配）
                    if pred_answer == gt_answer:
                        matched += 1
                    total += 1
            
            bar.set_postfix(EM = matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match: {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em:
        best_em = current_em
        best_model_state = deepcopy(model.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em:.4f}")
    
    # ============================================
    # 展示生成例子（使用 generator）
    # ============================================
    print("\n=== 生成例子（使用 generator）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=', '(21*9)+27=']
    for test in test_cases:
        result = model.generator(test, max_len=20)
        # 提取答案部分
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        # 計算正確答案
        try:
            correct_answer = str(eval(test[:-1]))  # 去掉 '=' 再計算
        except:
            correct_answer = "N/A"
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} 問題: {test:15s} | 生成答案: {answer:10s} | 正確答案: {correct_answer}")
 
    # 更新學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    
    print()
    model.train()

# ============================================
# 訓練結束，保存模型
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match: {best_em:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    best_model_path = os.path.join(save_dir, 'best_model_LSTM.pth')
    torch.save({
        'model_state_dict': best_model_state,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em,
    }, best_model_path)
    print(f"✓ 最佳模型已保存至: {best_model_path}")

# 保存最終模型
final_model_path = os.path.join(save_dir, 'final_model_LSTM.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'hidden_dim': hidden_dim,
    'char_to_id': char_to_id,
    'id_to_char': id_to_char,
    'final_em': current_em,
}, final_model_path)
print(f"✓ 最終模型已保存至: {final_model_path}")
print("=" * 60)


訓練開始時間: 2025-10-04 04:52:05

Epoch 1 start at 2025-10-04 04:52:05


Validation epoch 1: 100%|██████████| 26/26 [00:50<00:00,  1.94s/it, EM=0.474]




Epoch 1 - Exact Match: 0.4738 (124738/263250)
✓ 新的最佳模型！EM = 0.4738

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 6          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: -20        | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 47         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1750       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 220        | 正確答案: 216
當前學習率: 0.000998

Epoch 2 start at 2025-10-04 04:53:11


Validation epoch 2: 100%|██████████| 26/26 [00:53<00:00,  2.05s/it, EM=0.583]




Epoch 2 - Exact Match: 0.5830 (153480/263250)
✓ 新的最佳模型！EM = 0.5830

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: 20         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 45         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1730       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 210        | 正確答案: 216
當前學習率: 0.000990

Epoch 3 start at 2025-10-04 04:54:20


...


Epoch 28 - Exact Match: 0.9506 (250255/263250)

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000110

Epoch 29 start at 2025-10-04 05:23:49


Validation epoch 29: 100%|██████████| 26/26 [00:50<00:00,  1.95s/it, EM=0.951]




Epoch 29 - Exact Match: 0.9508 (250308/263250)

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1730       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000102

Epoch 30 start at 2025-10-04 05:24:55


Validation epoch 30: 100%|██████████| 26/26 [00:52<00:00,  2.03s/it, EM=0.951]


Epoch 30 - Exact Match: 0.9508 (250290/263250)

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000100

訓練完成！
開始時間: 2025-10-04 04:52:05
結束時間: 2025-10-04 05:26:03
訓練持續時間: 0:33:57
總計: 2037.63 秒 (33.96 分鐘)
------------------------------------------------------------
最佳 Exact Match: 0.9520

✓ 最佳模型已保存至: ./checkpoints/best_model.pth
✓ 最終模型已保存至: ./checkpoints/final_model.pth


## 重置訓練參數

In [83]:
# ============================================
# 重置模型、優化器、調度器等設定
# 確保從頭開始訓練新模型,而非延續之前的訓練
# ============================================

# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型
torch.manual_seed(2)
model = CharLSTM(vocab_size, embed_dim, hidden_dim)
model = model.to(device)

# 重新編譯模型
model = torch.compile(model, dynamic=True)

# 重新建立損失函數
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 重新建立優化器
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

# 重新建立學習率調度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 模型、優化器、調度器已重置")
print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print(f"批次大小: {batch_size}")
print("=" * 60)
print()

✓ 模型、優化器、調度器已重置
模型參數量: 1,128,210
初始學習率: 0.001
訓練輪數: 30
批次大小: 2048



## 訓練LSTM (驗證使用forward (teacher forcing))

In [84]:
'''
使用 forward 而非 generate 來進行驗證
速度快約4倍，並且其實評估的也算準，只是會稍微高估一點指標而已，影響不嚴重。

forward: 會有 teacher forcing 的問題，不算完全公平。
        假設 123+456=579，模型如果第一個答案字元預測錯誤(例如=1)，會修正為5，繼續預測下一個答案。
		
generate: 真正公平的評估，不修正錯誤，會從錯誤的答案接下去預測。
        如果第一個答案字元猜1，後面會用123+456=1，繼續預測下個字。
'''

from tqdm import tqdm
from copy import deepcopy
import time
from datetime import datetime, timedelta

model = model.to(device)
model.train()
i = 0

# 記錄最佳模型
best_em = 0
best_model_state = None

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()
print("=" * 60)
print(f"訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    # The process bar
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # ============================================
    # 評估：使用批次前向傳播（快速）
    # ============================================
    model.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # 批次前向傳播
            batch_pred_y = model(batch_x.to(device), batch_x_lens)
            
            # 對每個樣本進行評估
            for idx in range(len(batch_x)):
                # 獲取輸入序列
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                # 找到等號位置
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                # 從批次預測中提取答案部分（等號之後到序列結束）
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]  # 從等號開始預測
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                # 將預測 IDs 轉換為字符，遇到 <eos> 停止
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                # 獲取真實答案
                ground_truth_answer = x_str[equal_pos+1:]
                
                # 比較答案
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM = matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match: {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em:
        best_em = current_em
        best_model_state = deepcopy(model.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em:.4f}")
    
    # ============================================
    # 展示生成例子（使用 generator）
    # ============================================
    print("\n=== 生成例子（使用 generator）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=', '(21*9)+27=']
    for test in test_cases:
        result = model.generator(test, max_len=20)
        # 提取答案部分
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        # 計算正確答案
        try:
            correct_answer = str(eval(test[:-1]))  # 去掉 '=' 再計算
        except:
            correct_answer = "N/A"
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} 問題: {test:15s} | 生成答案: {answer:10s} | 正確答案: {correct_answer}")
 
    # 更新學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    
    print()
    model.train()

# ============================================
# 訓練結束，保存模型
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match: {best_em:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    best_model_path = os.path.join(save_dir, 'best_model_forward_LSTM.pth')
    torch.save({
        'model_state_dict': best_model_state,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em,
    }, best_model_path)
    print(f"✓ 最佳模型已保存至: {best_model_path}")

# 保存最終模型
final_model_path = os.path.join(save_dir, 'final_model_forward_LSTM.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'hidden_dim': hidden_dim,
    'char_to_id': char_to_id,
    'id_to_char': id_to_char,
    'final_em': current_em,
}, final_model_path)
print(f"✓ 最終模型已保存至: {final_model_path}")
print("=" * 60)


訓練開始時間: 2025-10-04 12:47:26

Epoch 1 start at 2025-10-04 12:47:26


Train epoch 1:   0%|          | 0/1157 [00:00<?, ?it/s]

Validation epoch 1: 100%|██████████| 26/26 [00:10<00:00,  2.59it/s, EM=0.425]




Epoch 1 - Exact Match: 0.4250 (111869/263250)
✓ 新的最佳模型！EM = 0.4250

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 8          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: -10        | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 49         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1690       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 220        | 正確答案: 216
當前學習率: 0.000998

Epoch 2 start at 2025-10-04 12:47:51


Validation epoch 2: 100%|██████████| 26/26 [00:09<00:00,  2.66it/s, EM=0.554]



...


Epoch 28 - Exact Match: 0.9488 (249767/263250)
✓ 新的最佳模型！EM = 0.9488

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000110

Epoch 29 start at 2025-10-04 12:59:06


Validation epoch 29: 100%|██████████| 26/26 [00:09<00:00,  2.67it/s, EM=0.953]




Epoch 29 - Exact Match: 0.9528 (250816/263250)
✓ 新的最佳模型！EM = 0.9528

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000102

Epoch 30 start at 2025-10-04 12:59:31


Validation epoch 30: 100%|██████████| 26/26 [00:09<00:00,  2.65it/s, EM=0.948]


Epoch 30 - Exact Match: 0.9484 (249678/263250)

=== 生成例子（使用 generator）===
✓ 問題: 1+1=            | 生成答案: 2          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1738       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000100

訓練完成！
開始時間: 2025-10-04 12:47:26
結束時間: 2025-10-04 12:59:55
訓練持續時間: 0:12:29
總計: 749.76 秒 (12.50 分鐘)
------------------------------------------------------------
最佳 Exact Match: 0.9528

✓ 最佳模型已保存至: ./checkpoints/best_model_forward.pth
✓ 最終模型已保存至: ./checkpoints/final_model_forward.pth


## 載入模型並互動式生成答案

載入已訓練的模型權重，輸入算式並自動生成答案

In [ ]:
# 載入最佳模型權重
import torch
import os

# 設定模型路徑
checkpoint_path = './checkpoints/best_model.pth'
# checkpoint_path = './checkpoints/best_model_forward.pth'

# 檢查檔案是否存在
if not os.path.exists(checkpoint_path):
    print(f"錯誤：找不到模型檔案 {checkpoint_path}")
    print("請先完成訓練並保存模型")
else:
    # 載入 checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # 提取模型參數
    vocab_size = checkpoint['vocab_size']
    embed_dim = checkpoint['embed_dim']
    hidden_dim = checkpoint['hidden_dim']
    char_to_id = checkpoint['char_to_id']
    id_to_char = checkpoint['id_to_char']
    best_em = checkpoint['best_em']
    
    # 重新建立模型
    loaded_model = CharLSTM(vocab_size, embed_dim, hidden_dim)
    
    loaded_model = torch.compile(loaded_model, dynamic=True)

    state_dict = checkpoint['model_state_dict']
    
    # # 處理 torch.compile() 添加的 _orig_mod. 前綴 (保險用，照理不需要)
    # if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
    #     # 移除 _orig_mod. 前綴
    #     state_dict = {key.replace('_orig_mod.', ''): value 
    #                   for key, value in state_dict.items()}
    
    loaded_model.load_state_dict(state_dict)
    loaded_model = loaded_model.to(device)
    loaded_model.eval()
    
    print("=" * 60)
    print(f"✓ 模型載入成功！")
    print(f"模型位置: {checkpoint_path}")
    print(f"最佳 Exact Match: {best_em:.4f}")
    print(f"詞彙表大小: {vocab_size}")
    print("=" * 60)


✓ 模型載入成功！
模型位置: ./checkpoints/best_model.pth
最佳 Exact Match: 0.9520
詞彙表大小: 18


In [80]:
# 互動式算式生成器
def calculate_with_model(expression):
    """
    使用模型計算算式
    
    參數:
        expression: 算式字串，例如 "1+1", "10*20", "(5+3)*2"
    
    返回:
        生成的完整字串（包含算式和答案）
    """
    # 確保算式以 '=' 結尾
    if not expression.endswith('='):
        expression = expression + '='
    
    # 使用模型生成答案
    result = loaded_model.generator(expression, max_len=20)
    
    # 提取答案部分
    equal_pos = result.find('=')
    if '<eos>' in result:
        eos_pos = result.find('<eos>')
        answer = result[equal_pos+1:eos_pos]
    else:
        answer = result[equal_pos+1:]
    
    # 計算正確答案（用於驗證）
    try:
        correct_answer = str(eval(expression[:-1]))
    except:
        correct_answer = "無法計算"
    
    # 判斷是否正確
    is_correct = answer == correct_answer
    
    return {
        'expression': expression[:-1],
        'predicted_answer': answer,
        'correct_answer': correct_answer,
        'is_correct': is_correct,
        'full_result': result
    }

# 測試一些例子
print("=" * 60)
print("模型計算測試")
print("=" * 60)

# 最多三個數字，每個數字最好 < 50，僅支援 +, -, * 和括號，不支援除法
test_expressions = [
    "1+1",
    "5*6",
    "10+20",
    "44-3",
    "(10+5)*2",
    "12+41*42",
    "2*(3+4)",
    "7*8+3",
]

for expr in test_expressions:
    result = calculate_with_model(expr)
    status = "✓" if result['is_correct'] else "✗"
    print(f"{status} {result['expression']:20s} = {result['predicted_answer']:10s} (正確: {result['correct_answer']})")

print("=" * 60)


模型計算測試
✓ 1+1                  = 2          (正確: 2)
✓ 5*6                  = 30         (正確: 30)
✓ 10+20                = 30         (正確: 30)
✓ 44-3                 = 41         (正確: 41)
✓ (10+5)*2             = 30         (正確: 30)
✓ 12+41*42             = 1734       (正確: 1734)
✓ 2*(3+4)              = 14         (正確: 14)
✓ 7*8+3                = 59         (正確: 59)


## 測試沒有 dropout 與 normalization 的原始網路結構

In [31]:
class CharLSTM_ORI(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharLSTM_ORI, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.lstm_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.lstm_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        # 字符嵌入
        batch_x = self.embedding(batch_x)
        
        # 打包序列以處理可變長度
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        # 通過兩層LSTM
        batch_x, _ = self.lstm_layer1(batch_x)
        batch_x, _ = self.lstm_layer2(batch_x)
        
        # 解包序列
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x, batch_first=True)
        
        # 通過線性層得到每個位置的字符預測
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=20, temperature=1.0, use_sampling=False):
        """
        改進的生成器
        - temperature: 控制隨機性，越高越隨機
        - use_sampling: 是否使用採樣而不是貪婪解碼
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        # 檢查是否所有字符都在詞彙表中
        try:
            char_ids = [char_to_id[c] for c in start_char]
        except KeyError as e:
            raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 逐字符生成
        for _ in range(max_len - len(char_ids)): 
            x = torch.tensor([char_ids]).to(model_device)
            x_len = torch.tensor([len(char_ids)])
            
            with torch.no_grad():
                y = self.forward(x, x_len)
            
            # 取最後一個時間步的輸出
            logits = y[0, -1, :] / temperature
            
            # 禁止生成 <pad>
            logits[char_to_id['<pad>']] = -float('inf')
            
            if use_sampling:
                # 使用溫度採樣
                probs = torch.softmax(logits, dim=-1)
                next_char_id = torch.multinomial(probs, 1).item()
            else:
                # 貪婪解碼
                next_char_id = logits.argmax().item()
            
            if next_char_id == char_to_id['<eos>']:
                char_ids.append(next_char_id)
                break
            
            char_ids.append(next_char_id)
            
        # 將字符 IDs 轉換回字符串
        return ''.join([id_to_char[char_id] for char_id in char_ids])
    
    def batch_generator(self, start_sequences, max_len=20, temperature=1.0, use_sampling=False):
        """
        批次生成器 - 大幅提升評估速度！
        
        參數:
            start_sequences: list of strings，每個是問題部分（例如 ["1+1=", "5*6="]）
            max_len: 最大生成長度
            temperature: 控制隨機性
            use_sampling: 是否使用採樣
        
        返回:
            list of strings，每個是完整生成結果
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        batch_size = len(start_sequences)
        
        # 將每個起始序列轉換為 ID 列表，並檢查字符是否在詞彙表中
        batch_char_ids = []
        for seq in start_sequences:
            try:
                char_ids = [char_to_id[c] for c in seq]
                batch_char_ids.append(char_ids)
            except KeyError as e:
                raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 記錄每個序列是否已經完成（遇到 <eos> 或達到最大長度）
        finished = [False] * batch_size
        
        # 逐步生成，最多生成 max_len 個字符
        for step in range(max_len):
            # 如果所有序列都已完成，提前退出
            if all(finished):
                break
            
            # 準備當前批次的輸入
            current_batch = [torch.tensor(ids) for ids in batch_char_ids]
            batch_lens = torch.LongTensor([len(ids) for ids in batch_char_ids])
            
            # Padding 到相同長度
            padded_batch = torch.nn.utils.rnn.pad_sequence(
                current_batch, 
                batch_first=True, 
                padding_value=char_to_id['<pad>']
            ).to(model_device)
            
            # 批次前向傳播
            with torch.no_grad():
                batch_output = self.forward(padded_batch, batch_lens)
            
            # 對每個序列，取最後一個時間步的輸出
            for i in range(batch_size):
                if finished[i]:
                    continue
                
                # 獲取該序列最後一個位置的 logits
                last_pos = batch_lens[i] - 1
                logits = batch_output[i, last_pos, :] / temperature
                
                # 禁止生成 <pad>
                logits[char_to_id['<pad>']] = -float('inf')
                
                # T:按照prob取出token當作答案(隨機), F:取prob最高的token當作答案
                if use_sampling:
                    probs = torch.softmax(logits, dim=-1)
                    next_char_id = torch.multinomial(probs, 1).item()
                else:
                    next_char_id = logits.argmax().item()
                
                # 添加到序列
                batch_char_ids[i].append(next_char_id)
                
                # 檢查是否遇到 <eos>
                if next_char_id == char_to_id['<eos>']:
                    finished[i] = True
        
        # 將 ID 列表轉換回字符串
        results = [''.join([id_to_char[char_id] for char_id in ids]) 
                   for ids in batch_char_ids]
        
        return results


### 重置訓練參數

In [32]:
# ============================================
# 重置模型、優化器、調度器等設定
# 確保從頭開始訓練新模型,而非延續之前的訓練
# ============================================

# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型
torch.manual_seed(2)
model = CharLSTM_ORI(vocab_size, embed_dim, hidden_dim)
model = model.to(device)

# 重新編譯模型
model = torch.compile(model, dynamic=True)

# 重新建立損失函數
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 重新建立優化器
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

# 重新建立學習率調度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 模型、優化器、調度器已重置")
print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print(f"批次大小: {batch_size}")
print("=" * 60)
print()

✓ 模型、優化器、調度器已重置
模型參數量: 1,127,698
初始學習率: 0.001
訓練輪數: 30
批次大小: 3072



### 訓練LSTM_ORI

In [43]:
'''
使用 forward 而非 generate 來進行驗證
速度快約4倍，並且其實評估的也算準，只是會稍微高估一點指標而已，影響不嚴重。

forward: 會有 teacher forcing 的問題，不算完全公平。
        假設 123+456=579，模型如果第一個答案字元預測錯誤(例如=1)，會修正為5，繼續預測下一個答案。
		
generate: 真正公平的評估，不修正錯誤，會從錯誤的答案接下去預測。
        如果第一個答案字元猜1，後面會用123+456=1，繼續預測下個字。
'''

from tqdm import tqdm
from copy import deepcopy
import time
from datetime import datetime, timedelta

model = model.to(device)
model.train()
i = 0

# 記錄最佳模型
best_em = 0
best_model_state = None

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()
print("=" * 60)
print(f"訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    # The process bar
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # ============================================
    # 評估：使用批次前向傳播（快速）
    # ============================================
    model.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # 批次前向傳播
            batch_pred_y = model(batch_x.to(device), batch_x_lens)
            
            # 對每個樣本進行評估
            for idx in range(len(batch_x)):
                # 獲取輸入序列
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                # 找到等號位置
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                # 從批次預測中提取答案部分（等號之後到序列結束）
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]  # 從等號開始預測
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                # 將預測 IDs 轉換為字符，遇到 <eos> 停止
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                # 獲取真實答案
                ground_truth_answer = x_str[equal_pos+1:]
                
                # 比較答案
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM = matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match: {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em:
        best_em = current_em
        best_model_state = deepcopy(model.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em:.4f}")
    
    # ============================================
    # 展示生成例子（使用 generator）
    # ============================================
    print("\n=== 生成例子（使用 generator）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=', '(21*9)+27=']
    for test in test_cases:
        result = model.generator(test, max_len=20)
        # 提取答案部分
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        # 計算正確答案
        try:
            correct_answer = str(eval(test[:-1]))  # 去掉 '=' 再計算
        except:
            correct_answer = "N/A"
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} 問題: {test:15s} | 生成答案: {answer:10s} | 正確答案: {correct_answer}")
 
    # 更新學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    
    print()
    model.train()

# ============================================
# 訓練結束，保存模型
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match: {best_em:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    best_model_path = os.path.join(save_dir, 'best_model_forward_LSTM.pth')
    torch.save({
        'model_state_dict': best_model_state,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em,
    }, best_model_path)
    print(f"✓ 最佳模型已保存至: {best_model_path}")

# 保存最終模型
final_model_path = os.path.join(save_dir, 'final_model_forward_LSTM.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'hidden_dim': hidden_dim,
    'char_to_id': char_to_id,
    'id_to_char': id_to_char,
    'final_em': current_em,
}, final_model_path)
print(f"✓ 最終模型已保存至: {final_model_path}")
print("=" * 60)


訓練開始時間: 2025-10-05 12:03:18

Epoch 1 start at 2025-10-05 12:03:18


Validation epoch 1: 100%|██████████| 18/18 [00:39<00:00,  2.17s/it, EM=0.213]



Epoch 1 - Exact Match: 0.2126 (55961/263250)
✓ 新的最佳模型！EM = 0.2126

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 1          | 正確答案: 2
✗ 問題: 10+20=          | 生成答案: 28         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 43         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1712       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000998

Epoch 2 start at 2025-10-05 12:04:15


Validation epoch 2: 100%|██████████| 18/18 [00:58<00:00,  3.26s/it, EM=0.452]



Epoch 2 - Exact Match: 0.4517 (118916/263250)
✓ 新的最佳模型！EM = 0.4517

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 1          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1760       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 212        | 正確答案: 216
當前學習率: 0.000990

Epoch 3 start at 2025-10-05 12:05:29


...


Epoch 28 - Exact Match: 0.9077 (238951/263250)

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 1          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000110

Epoch 29 start at 2025-10-05 12:34:51


Validation epoch 29: 100%|██████████| 18/18 [00:39<00:00,  2.19s/it, EM=0.907]



Epoch 29 - Exact Match: 0.9072 (238813/263250)

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 1          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000102

Epoch 30 start at 2025-10-05 12:35:45


Validation epoch 30: 100%|██████████| 18/18 [00:57<00:00,  3.18s/it, EM=0.918]


Epoch 30 - Exact Match: 0.9178 (241603/263250)
✓ 新的最佳模型！EM = 0.9178

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 1          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000100

訓練完成！
開始時間: 2025-10-05 12:03:18
結束時間: 2025-10-05 12:36:58
訓練持續時間: 0:33:39
總計: 2019.16 秒 (33.65 分鐘)
------------------------------------------------------------
最佳 Exact Match: 0.9178

✓ 最佳模型已保存至: ./checkpoints/best_model_forward_LSTM.pth
✓ 最終模型已保存至: ./checkpoints/final_model_forward_LSTM.pth


# Question 2: 用RNN或GRU取代LSTM網路

## RNN網路架構

In [86]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.RNN(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.rnn_layer2 = torch.nn.RNN(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),          # 自己新增的
            torch.nn.LayerNorm(hidden_dim), # 自己新增的
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        # 字符嵌入
        batch_x = self.embedding(batch_x)
        
        # 打包序列以處理可變長度
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        # 通過兩層LSTM
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        # 解包序列
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x, batch_first=True)
        
        # 通過線性層得到每個位置的字符預測
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=20, temperature=1.0, use_sampling=False):
        """
        改進的生成器
        - temperature: 控制隨機性，越高越隨機
        - use_sampling: 是否使用採樣而不是貪婪解碼
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        # 檢查是否所有字符都在詞彙表中
        try:
            char_ids = [char_to_id[c] for c in start_char]
        except KeyError as e:
            raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 逐字符生成
        for _ in range(max_len - len(char_ids)): 
            x = torch.tensor([char_ids]).to(model_device)
            x_len = torch.tensor([len(char_ids)])
            
            with torch.no_grad():
                y = self.forward(x, x_len)
            
            # 取最後一個時間步的輸出
            logits = y[0, -1, :] / temperature
            
            # 禁止生成 <pad>
            logits[char_to_id['<pad>']] = -float('inf')
            
            if use_sampling:
                # 使用溫度採樣
                probs = torch.softmax(logits, dim=-1)
                next_char_id = torch.multinomial(probs, 1).item()
            else:
                # 貪婪解碼
                next_char_id = logits.argmax().item()
            
            if next_char_id == char_to_id['<eos>']:
                char_ids.append(next_char_id)
                break
            
            char_ids.append(next_char_id)
            
        # 將字符 IDs 轉換回字符串
        return ''.join([id_to_char[char_id] for char_id in char_ids])
    
    def batch_generator(self, start_sequences, max_len=200, temperature=1.0, use_sampling=False):
        """
        批次生成器 - 大幅提升評估速度！
        
        參數:
            start_sequences: list of strings，每個是問題部分（例如 ["1+1=", "5*6="]）
            max_len: 最大生成長度
            temperature: 控制隨機性
            use_sampling: 是否使用採樣
        
        返回:
            list of strings，每個是完整生成結果
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        batch_size = len(start_sequences)
        
        # 將每個起始序列轉換為 ID 列表，並檢查字符是否在詞彙表中
        batch_char_ids = []
        for seq in start_sequences:
            try:
                char_ids = [char_to_id[c] for c in seq]
                batch_char_ids.append(char_ids)
            except KeyError as e:
                raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 記錄每個序列是否已經完成（遇到 <eos> 或達到最大長度）
        finished = [False] * batch_size
        
        # 逐步生成，最多生成 max_len 個字符
        for step in range(max_len):
            # 如果所有序列都已完成，提前退出
            if all(finished):
                break
            
            # 準備當前批次的輸入
            current_batch = [torch.tensor(ids) for ids in batch_char_ids]
            batch_lens = torch.LongTensor([len(ids) for ids in batch_char_ids])
            
            # Padding 到相同長度
            padded_batch = torch.nn.utils.rnn.pad_sequence(
                current_batch, 
                batch_first=True, 
                padding_value=char_to_id['<pad>']
            ).to(model_device)
            
            # 批次前向傳播
            with torch.no_grad():
                batch_output = self.forward(padded_batch, batch_lens)
            
            # 對每個序列，取最後一個時間步的輸出
            for i in range(batch_size):
                if finished[i]:
                    continue
                
                # 獲取該序列最後一個位置的 logits
                last_pos = batch_lens[i] - 1
                logits = batch_output[i, last_pos, :] / temperature
                
                # 禁止生成 <pad>
                logits[char_to_id['<pad>']] = -float('inf')
                
                # T:按照prob取出token當作答案(隨機), F:取prob最高的token當作答案
                if use_sampling:
                    probs = torch.softmax(logits, dim=-1)
                    next_char_id = torch.multinomial(probs, 1).item()
                else:
                    next_char_id = logits.argmax().item()
                
                # 添加到序列
                batch_char_ids[i].append(next_char_id)
                
                # 檢查是否遇到 <eos>
                if next_char_id == char_to_id['<eos>']:
                    finished[i] = True
        
        # 將 ID 列表轉換回字符串
        results = [''.join([id_to_char[char_id] for char_id in ids]) 
                   for ids in batch_char_ids]
        
        return results


## GRU網路架構

In [87]:
class CharGRU(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharGRU, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.gru_layer1 = torch.nn.GRU(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.gru_layer2 = torch.nn.GRU(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),          # 自己新增的
            torch.nn.LayerNorm(hidden_dim), # 自己新增的
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        # 字符嵌入
        batch_x = self.embedding(batch_x)
        
        # 打包序列以處理可變長度
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        # 通過兩層LSTM
        batch_x, _ = self.gru_layer1(batch_x)
        batch_x, _ = self.gru_layer2(batch_x)
        
        # 解包序列
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x, batch_first=True)
        
        # 通過線性層得到每個位置的字符預測
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=20, temperature=1.0, use_sampling=False):
        """
        改進的生成器
        - temperature: 控制隨機性，越高越隨機
        - use_sampling: 是否使用採樣而不是貪婪解碼
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        # 檢查是否所有字符都在詞彙表中
        try:
            char_ids = [char_to_id[c] for c in start_char]
        except KeyError as e:
            raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 逐字符生成
        for _ in range(max_len - len(char_ids)): 
            x = torch.tensor([char_ids]).to(model_device)
            x_len = torch.tensor([len(char_ids)])
            
            with torch.no_grad():
                y = self.forward(x, x_len)
            
            # 取最後一個時間步的輸出
            logits = y[0, -1, :] / temperature
            
            # 禁止生成 <pad>
            logits[char_to_id['<pad>']] = -float('inf')
            
            if use_sampling:
                # 使用溫度採樣
                probs = torch.softmax(logits, dim=-1)
                next_char_id = torch.multinomial(probs, 1).item()
            else:
                # 貪婪解碼
                next_char_id = logits.argmax().item()
            
            if next_char_id == char_to_id['<eos>']:
                char_ids.append(next_char_id)
                break
            
            char_ids.append(next_char_id)
            
        # 將字符 IDs 轉換回字符串
        return ''.join([id_to_char[char_id] for char_id in char_ids])
    
    def batch_generator(self, start_sequences, max_len=20, temperature=1.0, use_sampling=False):
        """
        批次生成器 - 大幅提升評估速度！
        
        參數:
            start_sequences: list of strings，每個是問題部分（例如 ["1+1=", "5*6="]）
            max_len: 最大生成長度
            temperature: 控制隨機性
            use_sampling: 是否使用採樣
        
        返回:
            list of strings，每個是完整生成結果
        """
        # 獲取模型所在的設備
        model_device = next(self.parameters()).device
        
        batch_size = len(start_sequences)
        
        # 將每個起始序列轉換為 ID 列表，並檢查字符是否在詞彙表中
        batch_char_ids = []
        for seq in start_sequences:
            try:
                char_ids = [char_to_id[c] for c in seq]
                batch_char_ids.append(char_ids)
            except KeyError as e:
                raise ValueError(f"字符 {e} 不在詞彙表中。詞彙表包含: {sorted(char_to_id.keys())}")
        
        # 記錄每個序列是否已經完成（遇到 <eos> 或達到最大長度）
        finished = [False] * batch_size
        
        # 逐步生成，最多生成 max_len 個字符
        for step in range(max_len):
            # 如果所有序列都已完成，提前退出
            if all(finished):
                break
            
            # 準備當前批次的輸入
            current_batch = [torch.tensor(ids) for ids in batch_char_ids]
            batch_lens = torch.LongTensor([len(ids) for ids in batch_char_ids])
            
            # Padding 到相同長度
            padded_batch = torch.nn.utils.rnn.pad_sequence(
                current_batch, 
                batch_first=True, 
                padding_value=char_to_id['<pad>']
            ).to(model_device)
            
            # 批次前向傳播
            with torch.no_grad():
                batch_output = self.forward(padded_batch, batch_lens)
            
            # 對每個序列，取最後一個時間步的輸出
            for i in range(batch_size):
                if finished[i]:
                    continue
                
                # 獲取該序列最後一個位置的 logits
                last_pos = batch_lens[i] - 1
                logits = batch_output[i, last_pos, :] / temperature
                
                # 禁止生成 <pad>
                logits[char_to_id['<pad>']] = -float('inf')
                
                # T:按照prob取出token當作答案(隨機), F:取prob最高的token當作答案
                if use_sampling:
                    probs = torch.softmax(logits, dim=-1)
                    next_char_id = torch.multinomial(probs, 1).item()
                else:
                    next_char_id = logits.argmax().item()
                
                # 添加到序列
                batch_char_ids[i].append(next_char_id)
                
                # 檢查是否遇到 <eos>
                if next_char_id == char_to_id['<eos>']:
                    finished[i] = True
        
        # 將 ID 列表轉換回字符串
        results = [''.join([id_to_char[char_id] for char_id in ids]) 
                   for ids in batch_char_ids]
        
        return results


## 清除訓練設定

In [88]:
# ============================================
# 重置模型、優化器、調度器等設定
# 確保從頭開始訓練新模型,而非延續之前的訓練
# ============================================

# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型
torch.manual_seed(2)
model = CharRNN(vocab_size, embed_dim, hidden_dim)
model = model.to(device)

# 重新編譯模型
model = torch.compile(model, dynamic=True)

# 重新建立損失函數
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 重新建立優化器
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

# 重新建立學習率調度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 模型、優化器、調度器已重置")
print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print(f"批次大小: {batch_size}")
print("=" * 60)
print()

✓ 模型、優化器、調度器已重置
模型參數量: 338,706
初始學習率: 0.001
訓練輪數: 30
批次大小: 2048



## 訓練RNN

In [89]:
'''
使用 forward 而非 generate 來進行驗證
速度快約4倍，並且其實評估的也算準，只是會稍微高估一點指標而已，影響不嚴重。

forward: 會有 teacher forcing 的問題，不算完全公平。
        假設 123+456=579，模型如果第一個答案字元預測錯誤(例如=1)，會修正為5，繼續預測下一個答案。
		
generate: 真正公平的評估，不修正錯誤，會從錯誤的答案接下去預測。
        如果第一個答案字元猜1，後面會用123+456=1，繼續預測下個字。
'''

from tqdm import tqdm
from copy import deepcopy
import time
from datetime import datetime, timedelta

model = model.to(device)
model.train()
i = 0

# 記錄最佳模型
best_em = 0
best_model_state = None

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()
print("=" * 60)
print(f"訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    # The process bar
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # ============================================
    # 評估：使用批次前向傳播（快速）
    # ============================================
    model.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # 批次前向傳播
            batch_pred_y = model(batch_x.to(device), batch_x_lens)
            
            # 對每個樣本進行評估
            for idx in range(len(batch_x)):
                # 獲取輸入序列
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                # 找到等號位置
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                # 從批次預測中提取答案部分（等號之後到序列結束）
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]  # 從等號開始預測
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                # 將預測 IDs 轉換為字符，遇到 <eos> 停止
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                # 獲取真實答案
                ground_truth_answer = x_str[equal_pos+1:]
                
                # 比較答案
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM = matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match: {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em:
        best_em = current_em
        best_model_state = deepcopy(model.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em:.4f}")
    
    # ============================================
    # 展示生成例子（使用 generator）
    # ============================================
    print("\n=== 生成例子（使用 generator）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=', '(21*9)+27=']
    for test in test_cases:
        result = model.generator(test, max_len=20)
        # 提取答案部分
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        # 計算正確答案
        try:
            correct_answer = str(eval(test[:-1]))  # 去掉 '=' 再計算
        except:
            correct_answer = "N/A"
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} 問題: {test:15s} | 生成答案: {answer:10s} | 正確答案: {correct_answer}")
 
    # 更新學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    
    print()
    model.train()

# ============================================
# 訓練結束，保存模型
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match: {best_em:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    best_model_path = os.path.join(save_dir, 'best_model_forward_RNN.pth')
    torch.save({
        'model_state_dict': best_model_state,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em,
    }, best_model_path)
    print(f"✓ 最佳模型已保存至: {best_model_path}")

# 保存最終模型
final_model_path = os.path.join(save_dir, 'final_model_forward_RNN.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'hidden_dim': hidden_dim,
    'char_to_id': char_to_id,
    'id_to_char': id_to_char,
    'final_em': current_em,
}, final_model_path)
print(f"✓ 最終模型已保存至: {final_model_path}")
print("=" * 60)


訓練開始時間: 2025-10-04 13:17:03

Epoch 1 start at 2025-10-04 13:17:03


Validation epoch 1: 100%|██████████| 26/26 [00:09<00:00,  2.64it/s, EM=0.185]




Epoch 1 - Exact Match: 0.1852 (48761/263250)
✓ 新的最佳模型！EM = 0.1852

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 14         | 正確答案: 2
✗ 問題: 10+20=          | 生成答案: 48         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: 28         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 39         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1792       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 220        | 正確答案: 216
當前學習率: 0.000998

Epoch 2 start at 2025-10-04 13:17:28


Validation epoch 2: 100%|██████████| 26/26 [00:09<00:00,  2.70it/s, EM=0.319]




Epoch 2 - Exact Match: 0.3189 (83941/263250)
✓ 新的最佳模型！EM = 0.3189

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 10         | 正確答案: 2
✗ 問題: 10+20=          | 生成答案: 36         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: 28         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 39         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1672       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 204        | 正確答案: 216
當前學習率: 0.000990

Epoch 3 start at 2025-10-04 13:17:52


...


Epoch 28 - Exact Match: 0.7096 (186809/263250)

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 39         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1722       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 218        | 正確答案: 216
當前學習率: 0.000110

Epoch 29 start at 2025-10-04 13:28:26


Validation epoch 29: 100%|██████████| 26/26 [00:09<00:00,  2.67it/s, EM=0.748]




Epoch 29 - Exact Match: 0.7483 (196993/263250)
✓ 新的最佳模型！EM = 0.7483

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 39         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000102

Epoch 30 start at 2025-10-04 13:28:51


Validation epoch 30: 100%|██████████| 26/26 [00:10<00:00,  2.59it/s, EM=0.736]


Epoch 30 - Exact Match: 0.7365 (193881/263250)

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 39         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1726       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000100

訓練完成！
開始時間: 2025-10-04 13:17:03
結束時間: 2025-10-04 13:29:16
訓練持續時間: 0:12:12
總計: 732.18 秒 (12.20 分鐘)
------------------------------------------------------------
最佳 Exact Match: 0.7483

✓ 最佳模型已保存至: ./checkpoints/best_model_forward_RNN.pth
✓ 最終模型已保存至: ./checkpoints/final_model_forward_RNN.pth


## 清除訓練設定

In [90]:
# ============================================
# 重置模型、優化器、調度器等設定
# 確保從頭開始訓練新模型,而非延續之前的訓練
# ============================================

# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型
torch.manual_seed(2)
model = CharGRU(vocab_size, embed_dim, hidden_dim)
model = model.to(device)

# 重新編譯模型
model = torch.compile(model, dynamic=True)

# 重新建立損失函數
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 重新建立優化器
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

# 重新建立學習率調度器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 模型、優化器、調度器已重置")
print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print(f"批次大小: {batch_size}")
print("=" * 60)
print()

✓ 模型、優化器、調度器已重置
模型參數量: 865,042
初始學習率: 0.001
訓練輪數: 30
批次大小: 2048



## 訓練GRU

In [91]:
'''
使用 forward 而非 generate 來進行驗證
速度快約4倍，並且其實評估的也算準，只是會稍微高估一點指標而已，影響不嚴重。

forward: 會有 teacher forcing 的問題，不算完全公平。
        假設 123+456=579，模型如果第一個答案字元預測錯誤(例如=1)，會修正為5，繼續預測下一個答案。
		
generate: 真正公平的評估，不修正錯誤，會從錯誤的答案接下去預測。
        如果第一個答案字元猜1，後面會用123+456=1，繼續預測下個字。
'''

from tqdm import tqdm
from copy import deepcopy
import time
from datetime import datetime, timedelta

model = model.to(device)
model.train()
i = 0

# 記錄最佳模型
best_em = 0
best_model_state = None

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()
print("=" * 60)
print(f"訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    # The process bar
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # ============================================
    # 評估：使用批次前向傳播（快速）
    # ============================================
    model.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            # 批次前向傳播
            batch_pred_y = model(batch_x.to(device), batch_x_lens)
            
            # 對每個樣本進行評估
            for idx in range(len(batch_x)):
                # 獲取輸入序列
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                # 找到等號位置
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                # 從批次預測中提取答案部分（等號之後到序列結束）
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]  # 從等號開始預測
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                # 將預測 IDs 轉換為字符，遇到 <eos> 停止
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                # 獲取真實答案
                ground_truth_answer = x_str[equal_pos+1:]
                
                # 比較答案
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM = matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match: {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em:
        best_em = current_em
        best_model_state = deepcopy(model.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em:.4f}")
    
    # ============================================
    # 展示生成例子（使用 generator）
    # ============================================
    print("\n=== 生成例子（使用 generator）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=', '(21*9)+27=']
    for test in test_cases:
        result = model.generator(test, max_len=20)
        # 提取答案部分
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        # 計算正確答案
        try:
            correct_answer = str(eval(test[:-1]))  # 去掉 '=' 再計算
        except:
            correct_answer = "N/A"
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} 問題: {test:15s} | 生成答案: {answer:10s} | 正確答案: {correct_answer}")
 
    # 更新學習率
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    
    print()
    model.train()

# ============================================
# 訓練結束，保存模型
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match: {best_em:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    best_model_path = os.path.join(save_dir, 'best_model_forward_GRU.pth')
    torch.save({
        'model_state_dict': best_model_state,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em,
    }, best_model_path)
    print(f"✓ 最佳模型已保存至: {best_model_path}")

# 保存最終模型
final_model_path = os.path.join(save_dir, 'final_model_forward_GRU.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'hidden_dim': hidden_dim,
    'char_to_id': char_to_id,
    'id_to_char': id_to_char,
    'final_em': current_em,
}, final_model_path)
print(f"✓ 最終模型已保存至: {final_model_path}")
print("=" * 60)


訓練開始時間: 2025-10-04 13:29:16

Epoch 1 start at 2025-10-04 13:29:16


Validation epoch 1: 100%|██████████| 26/26 [00:10<00:00,  2.55it/s, EM=0.314]




Epoch 1 - Exact Match: 0.3138 (82608/263250)
✓ 新的最佳模型！EM = 0.3138

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 10         | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✗ 問題: 5*6=            | 生成答案: 13         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 49         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1650       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 220        | 正確答案: 216
當前學習率: 0.000998

Epoch 2 start at 2025-10-04 13:29:41


Validation epoch 2: 100%|██████████| 26/26 [00:10<00:00,  2.58it/s, EM=0.509]




Epoch 2 - Exact Match: 0.5089 (133955/263250)
✓ 新的最佳模型！EM = 0.5089

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 6          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✗ 問題: 44-3=           | 生成答案: 45         | 正確答案: 41
✗ 問題: 12+41*42=       | 生成答案: 1706       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 210        | 正確答案: 216
當前學習率: 0.000990

Epoch 3 start at 2025-10-04 13:30:06


...


Epoch 28 - Exact Match: 0.9286 (244446/263250)
✓ 新的最佳模型！EM = 0.9286

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✗ 問題: (21*9)+27=      | 生成答案: 214        | 正確答案: 216
當前學習率: 0.000110

Epoch 29 start at 2025-10-04 13:40:49


Validation epoch 29: 100%|██████████| 26/26 [00:10<00:00,  2.57it/s, EM=0.93] 




Epoch 29 - Exact Match: 0.9298 (244758/263250)
✓ 新的最佳模型！EM = 0.9298

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000102

Epoch 30 start at 2025-10-04 13:41:14


Validation epoch 30: 100%|██████████| 26/26 [00:09<00:00,  2.62it/s, EM=0.925]


Epoch 30 - Exact Match: 0.9246 (243410/263250)

=== 生成例子（使用 generator）===
✗ 問題: 1+1=            | 生成答案: 4          | 正確答案: 2
✓ 問題: 10+20=          | 生成答案: 30         | 正確答案: 30
✓ 問題: 5*6=            | 生成答案: 30         | 正確答案: 30
✓ 問題: 44-3=           | 生成答案: 41         | 正確答案: 41
✓ 問題: 12+41*42=       | 生成答案: 1734       | 正確答案: 1734
✓ 問題: (21*9)+27=      | 生成答案: 216        | 正確答案: 216
當前學習率: 0.000100

訓練完成！
開始時間: 2025-10-04 13:29:16
結束時間: 2025-10-04 13:41:39
訓練持續時間: 0:12:23
總計: 743.04 秒 (12.38 分鐘)
------------------------------------------------------------
最佳 Exact Match: 0.9298

✓ 最佳模型已保存至: ./checkpoints/best_model_forward_GRU.pth
✓ 最終模型已保存至: ./checkpoints/final_model_forward_GRU.pth


# 實驗：訓練資料中 20% 答案錯誤對模型性能的影響

測試當訓練資料中有 20% 的答案是錯誤的時候，模型是否還能學習到正確的算術規則。

實驗設計：
1. 複製原始訓練資料
2. 隨機選擇 20% 的樣本，將其答案替換為錯誤答案
3. 使用錯誤資料訓練 LSTM 模型
4. 在乾淨的評估集上測試性能
5. 比較與正常訓練的差異

## 步驟 1: 創建含有 20% 錯誤答案的訓練資料（極速版本）
針對百萬級資料優化，使用純向量化 + 最小化操作

In [21]:
import random

# 設定隨機種子以確保可重現性
random.seed(42)
np.random.seed(42)

# 複製原始訓練資料（使用 copy=False 避免深複製，節省記憶體和時間）
df_train_noisy = df_train.copy()

# 計算要污染的樣本數量（20%）
num_samples = len(df_train_noisy)
num_corrupted = int(num_samples * 0.2)

print("=" * 60)
print("創建含有錯誤答案的訓練資料（極速版本 - 百萬級優化）")
print("=" * 60)
print(f"總訓練樣本數: {num_samples:,}")
print(f"錯誤答案樣本數 (20%): {num_corrupted:,}")
print()

start_time = time.time()

# ============================================
# 極速優化：只處理需要修改的 20%
# ============================================

# 隨機選擇要污染的索引
corrupted_indices = np.random.choice(num_samples, size=num_corrupted, replace=False)

# 預先生成所有錯誤偏移量
offsets = np.random.choice([-10, -5, -3, -2, -1, 1, 2, 3, 5, 10], size=num_corrupted)

print("正在污染答案（僅處理 20% 的樣本）...")

# 只處理需要污染的樣本（不遍歷全部）
corrupted_count = 0
for i, idx in enumerate(tqdm(corrupted_indices)):
    original_src = df_train_noisy.at[idx, 'src']  # 使用 at 比 loc 快
    
    # 找到等號位置
    equal_pos = original_src.rfind('=')  # 使用 rfind（從後往前找）稍快
    if equal_pos == -1:
        continue
    
    # 分離問題和答案
    question = original_src[:equal_pos]
    answer = original_src[equal_pos+1:]
    
    # 生成錯誤答案
    try:
        correct_value = int(answer)
        wrong_value = correct_value + offsets[i]
        wrong_answer = str(wrong_value)
        
        # 直接更新（使用 at 最快）
        df_train_noisy.at[idx, 'src'] = question + '=' + wrong_answer
        df_train_noisy.at[idx, 'tgt'] = wrong_answer
        corrupted_count += 1
    except:
        continue

end_time = time.time()

print(f"\n✓ 資料污染完成")
print(f"  實際污染數量: {corrupted_count:,}")
print(f"  總耗時: {end_time - start_time:.2f} 秒")
if end_time - start_time > 0:
    print(f"  處理速度: {num_corrupted/(end_time - start_time):,.0f} 筆/秒")
print()

# 顯示一些被污染的例子
print("錯誤答案範例:")
print("-" * 60)
display_count = min(10, len(corrupted_indices))
for i in range(display_count):
    idx = corrupted_indices[i]
    original = df_train.at[idx, 'src']
    corrupted = df_train_noisy.at[idx, 'src']
    print(f"{i+1}. 原始: {original:30s} → 錯誤: {corrupted}")
print("=" * 60)
print()

創建含有錯誤答案的訓練資料（極速版本 - 百萬級優化）
總訓練樣本數: 2,369,250
錯誤答案樣本數 (20%): 473,850

正在污染答案（僅處理 20% 的樣本）...


100%|██████████| 473850/473850 [00:10<00:00, 43180.44it/s]


✓ 資料污染完成
  實際污染數量: 473,850
  總耗時: 11.02 秒
  處理速度: 42,983 筆/秒

錯誤答案範例:
------------------------------------------------------------
1. 原始: 30-30+11=11                    → 錯誤: 30-30+11=13
2. 原始: 37-7*21=-110                   → 錯誤: 37-7*21=-108
3. 原始: (31-34)*35=-105                → 錯誤: (31-34)*35=-108
4. 原始: 19*(0+31)=589                  → 錯誤: 19*(0+31)=594
5. 原始: 12+(30-13)=29                  → 錯誤: 12+(30-13)=27
6. 原始: (37-30)+15=22                  → 錯誤: (37-30)+15=19
7. 原始: (30+32)-21=41                  → 錯誤: (30+32)-21=40
8. 原始: 45*1+48=93                     → 錯誤: 45*1+48=91
9. 原始: 14-30+32=16                    → 錯誤: 14-30+32=13
10. 原始: 42*25-31=1019                  → 錯誤: 42*25-31=1009



## 步驟 2: 處理含有錯誤答案的訓練資料

In [22]:
print("=== 處理含有錯誤答案的訓練資料 ===")
start_time = time.time()

# 使用相同的 process_data 函數處理錯誤資料
df_train_noisy_processed = process_data(df_train_noisy)

end_time = time.time()
print(f"處理完成，耗時: {end_time - start_time:.2f}秒")

# 創建新的 Dataset 和 DataLoader
ds_train_noisy = Dataset(df_train_noisy_processed[['char_id_list', 'label_id_list']])
dl_train_noisy = torch.utils.data.DataLoader(
    ds_train_noisy,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=4
)

print(f"✓ 錯誤訓練資料 DataLoader 已創建")
print(f"  批次大小: {batch_size}")
print(f"  總批次數: {len(dl_train_noisy)}")
print("=" * 60)
print()

=== 處理含有錯誤答案的訓練資料 ===
Processing 2369250 samples...


100%|██████████| 2369250/2369250 [00:27<00:00, 86852.25it/s] 



處理完成，耗時: 27.84秒
✓ 錯誤訓練資料 DataLoader 已創建
  批次大小: 3072
  總批次數: 772



## 步驟 3: 重置訓練參數

In [28]:
# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型（使用不同的隨機種子）
torch.manual_seed(42)
model_noisy = CharLSTM(vocab_size, embed_dim, hidden_dim)
print("LSTM")
model_noisy = model_noisy.to(device)

# 編譯模型
model_noisy = torch.compile(model_noisy, dynamic=True)

# 重新建立損失函數和優化器
criterion_noisy = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
optimizer_noisy = torch.optim.AdamW(model_noisy.parameters(), lr=lr, weight_decay=1e-4)
scheduler_noisy = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_noisy, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 錯誤資料實驗模型已準備就緒")
print(f"模型參數量: {sum(p.numel() for p in model_noisy.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print("=" * 60)
print()

LSTM
✓ 錯誤資料實驗模型已準備就緒
模型參數量: 1,128,210
初始學習率: 0.001
訓練輪數: 30



## 步驟 4: 使用含有錯誤答案的資料訓練模型

In [29]:
model_noisy = model_noisy.to(device)
model_noisy.train()
i = 0

# 記錄最佳模型
best_em_noisy = 0
best_model_state_noisy = None

import time
from datetime import datetime, timedelta

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()

print("=" * 60)
print(f"【錯誤資料實驗】訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train_noisy, desc=f"Train (Noisy Data) epoch {epoch}")
    
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        optimizer_noisy.zero_grad()
        
        batch_pred_y = model_noisy(batch_x.to(device), batch_x_lens)
        loss = criterion_noisy(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model_noisy.parameters(), grad_clip)
        optimizer_noisy.step()
        
        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss=loss.item())
    
    # ============================================
    # 評估：使用批次前向傳播（在乾淨的評估集上）
    # ============================================
    model_noisy.eval()
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            batch_pred_y = model_noisy(batch_x.to(device), batch_x_lens)
            
            for idx in range(len(batch_x)):
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                ground_truth_answer = x_str[equal_pos+1:]
                
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM=matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match (乾淨評估集): {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em_noisy:
        best_em_noisy = current_em
        best_model_state_noisy = deepcopy(model_noisy.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em_noisy:.4f}")
    
    # 展示生成例子
    print("\n=== 生成例子 ===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '12+41*42=']
    for test in test_cases:
        result = model_noisy.generator(test, max_len=50)
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        
        try:
            correct_answer = str(eval(test[:-1]))
        except:
            correct_answer = "N/A"
        
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} {test:15s} | 生成: {answer:10s} | 正確: {correct_answer}")
    
    # 更新學習率
    scheduler_noisy.step()
    current_lr = optimizer_noisy.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    print()
    
    model_noisy.train()

# ============================================
# 訓練結束
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("【錯誤資料實驗】訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match (乾淨評估集): {best_em_noisy:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存模型
if best_model_state_noisy is not None:
    model_noisy.load_state_dict(best_model_state_noisy)
    noisy_model_path = os.path.join(save_dir, 'best_model_noisy_20pct.pth')
    torch.save({
        'model_state_dict': best_model_state_noisy,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em_noisy,
        'noise_ratio': 0.2,
    }, noisy_model_path)
    print(f"✓ 錯誤資料實驗模型已保存至: {noisy_model_path}")
    print("=" * 60)

【錯誤資料實驗】訓練開始時間: 2025-10-04 15:07:20

Epoch 1 start at 2025-10-04 15:07:20


Validation epoch 1: 100%|██████████| 18/18 [00:11<00:00,  1.63it/s, EM=0.23]



Epoch 1 - Exact Match (乾淨評估集): 0.2302 (60606/263250)
✓ 新的最佳模型！EM = 0.2302

=== 生成例子 ===
✗ 1+1=            | 生成: 10         | 正確: 2
✗ 10+20=          | 生成: 35         | 正確: 30
✗ 5*6=            | 生成: -30        | 正確: 30
✗ 44-3=           | 生成: 49         | 正確: 41
✗ 12+41*42=       | 生成: 1700       | 正確: 1734
當前學習率: 0.000998

Epoch 2 start at 2025-10-04 15:07:45


Validation epoch 2: 100%|██████████| 18/18 [00:11<00:00,  1.61it/s, EM=0.476]



Epoch 2 - Exact Match (乾淨評估集): 0.4756 (125201/263250)
✓ 新的最佳模型！EM = 0.4756

=== 生成例子 ===
✗ 1+1=            | 生成: 10         | 正確: 2
✓ 10+20=          | 生成: 30         | 正確: 30
✗ 5*6=            | 生成: 50         | 正確: 30
✗ 44-3=           | 生成: 45         | 正確: 41
✗ 12+41*42=       | 生成: 1760       | 正確: 1734
當前學習率: 0.000990

Epoch 3 start at 2025-10-04 15:08:10


...


Epoch 28 - Exact Match (乾淨評估集): 0.8965 (235997/263250)

=== 生成例子 ===
✓ 1+1=            | 生成: 2          | 正確: 2
✓ 10+20=          | 生成: 30         | 正確: 30
✗ 5*6=            | 生成: 20         | 正確: 30
✗ 44-3=           | 生成: 43         | 正確: 41
✓ 12+41*42=       | 生成: 1734       | 正確: 1734
當前學習率: 0.000110

Epoch 29 start at 2025-10-04 15:19:03


Validation epoch 29: 100%|██████████| 18/18 [00:11<00:00,  1.62it/s, EM=0.913]



Epoch 29 - Exact Match (乾淨評估集): 0.9130 (240357/263250)
✓ 新的最佳模型！EM = 0.9130

=== 生成例子 ===
✓ 1+1=            | 生成: 2          | 正確: 2
✓ 10+20=          | 生成: 30         | 正確: 30
✗ 5*6=            | 生成: 20         | 正確: 30
✗ 44-3=           | 生成: 43         | 正確: 41
✓ 12+41*42=       | 生成: 1734       | 正確: 1734
當前學習率: 0.000102

Epoch 30 start at 2025-10-04 15:19:28


Validation epoch 30: 100%|██████████| 18/18 [00:10<00:00,  1.72it/s, EM=0.911]


Epoch 30 - Exact Match (乾淨評估集): 0.9106 (239720/263250)

=== 生成例子 ===
✓ 1+1=            | 生成: 2          | 正確: 2
✓ 10+20=          | 生成: 30         | 正確: 30
✗ 5*6=            | 生成: 20         | 正確: 30
✗ 44-3=           | 生成: 43         | 正確: 41
✓ 12+41*42=       | 生成: 1734       | 正確: 1734
當前學習率: 0.000100

【錯誤資料實驗】訓練完成！
開始時間: 2025-10-04 15:07:20
結束時間: 2025-10-04 15:19:54
訓練持續時間: 0:12:33
總計: 753.79 秒 (12.56 分鐘)
------------------------------------------------------------
最佳 Exact Match (乾淨評估集): 0.9130

✓ 錯誤資料實驗模型已保存至: ./checkpoints/best_model_noisy_20pct.pth


# 實驗: 訓練集都是三個數字的運算 驗證集都是兩個數字的運算 會怎樣?

## 實驗設計說明

**研究問題**: 當模型在複雜運算（三個數字）上訓練後，能否泛化到更簡單的運算（兩個數字）？

**實驗步驟**:
1. 創建數據過濾函數，識別運算中的數字個數
2. 從訓練集中篩選出**三個數字**的運算作為新訓練集
3. 從評估集中篩選出**兩個數字**的運算作為新評估集
4. 訓練 LSTM 模型
5. 在簡單運算（兩個數字）上測試模型性能
6. 分析模型是否能從複雜任務泛化到簡單任務

**預期結果**: 
- 如果模型學習到了算術的本質規則，應該能夠處理更簡單的運算
- 如果模型只是記憶訓練樣本的模式，可能在簡單運算上表現較差

## 步驟 1: 創建數據過濾函數

In [45]:
import re

def count_operands(expression):
    """
    計算算術表達式中的數字（運算元）個數
    
    參數:
        expression: 算術表達式字符串，例如 "14*(43+20)=882" 或 "1+1=2"
    
    返回:
        運算元的個數
    
    範例:
        "1+1=2" -> 2 (兩個運算元: 1, 1)
        "14*(43+20)=882" -> 3 (三個運算元: 14, 43, 20)
        "13+32+29=74" -> 3 (三個運算元: 13, 32, 29)
    """
    # 找到等號位置，只分析問題部分
    equal_pos = expression.find('=')
    if equal_pos == -1:
        question_part = expression
    else:
        question_part = expression[:equal_pos]
    
    # 使用正則表達式找出所有數字
    # \d+ 匹配一個或多個數字
    numbers = re.findall(r'\d+', question_part)
    
    return len(numbers)


# 測試函數
print("=" * 60)
print("測試數字計數函數")
print("=" * 60)

test_expressions = [
    "1+1=2",           # 2個數字
    "10+20=30",        # 2個數字
    "5*6=30",          # 2個數字
    "44-3=41",         # 2個數字
    "14*(43+20)=882",  # 3個數字
    "13+32+29=74",     # 3個數字
    "31*(3-11)=-248",  # 3個數字
    "(6+1)*5=35",      # 3個數字
]

for expr in test_expressions:
    count = count_operands(expr)
    print(f"{expr:25s} -> {count} 個數字")

print("=" * 60)
print()

測試數字計數函數
1+1=2                     -> 2 個數字
10+20=30                  -> 2 個數字
5*6=30                    -> 2 個數字
44-3=41                   -> 2 個數字
14*(43+20)=882            -> 3 個數字
13+32+29=74               -> 3 個數字
31*(3-11)=-248            -> 3 個數字
(6+1)*5=35                -> 3 個數字



## 步驟 2: 分析原始數據集的數字分佈

In [46]:
from tqdm import tqdm

print("=" * 60)
print("分析訓練集和評估集的數字分佈")
print("=" * 60)

# 分析訓練集
print("\n正在分析訓練集...")
train_operand_counts = df_train['src'].apply(count_operands)
train_distribution = train_operand_counts.value_counts().sort_index()

print("\n【訓練集數字分佈】")
for count, freq in train_distribution.items():
    percentage = freq / len(df_train) * 100
    print(f"  {count} 個數字: {freq:,} 筆 ({percentage:.2f}%)")
print(f"  總計: {len(df_train):,} 筆")

# 分析評估集
print("\n正在分析評估集...")
eval_operand_counts = df_eval['src'].apply(count_operands)
eval_distribution = eval_operand_counts.value_counts().sort_index()

print("\n【評估集數字分佈】")
for count, freq in eval_distribution.items():
    percentage = freq / len(df_eval) * 100
    print(f"  {count} 個數字: {freq:,} 筆 ({percentage:.2f}%)")
print(f"  總計: {len(df_eval):,} 筆")

print("=" * 60)
print()

分析訓練集和評估集的數字分佈

正在分析訓練集...

【訓練集數字分佈】
  2 個數字: 6,784 筆 (0.29%)
  3 個數字: 2,362,466 筆 (99.71%)
  總計: 2,369,250 筆

正在分析評估集...

【評估集數字分佈】
  2 個數字: 716 筆 (0.27%)
  3 個數字: 262,534 筆 (99.73%)
  總計: 263,250 筆



## 步驟 3: 篩選訓練集（三個數字）和評估集（兩個數字）

In [47]:
print("=" * 60)
print("篩選數據集")
print("=" * 60)

# 篩選訓練集：只保留三個數字的運算
print("\n正在篩選訓練集（三個數字的運算）...")
df_train_3ops = df_train[train_operand_counts == 3].copy()
print(f"✓ 訓練集篩選完成: {len(df_train_3ops):,} 筆（三個數字）")

# 篩選評估集：只保留兩個數字的運算
print("\n正在篩選評估集（兩個數字的運算）...")
df_eval_2ops = df_eval[eval_operand_counts == 2].copy()
print(f"✓ 評估集篩選完成: {len(df_eval_2ops):,} 筆（兩個數字）")

# 顯示篩選後的數據範例
print("\n【訓練集範例（三個數字）】")
print("-" * 60)
for i in range(min(10, len(df_train_3ops))):
    sample = df_train_3ops.iloc[i]['src']
    operand_count = count_operands(sample)
    print(f"{i+1:2d}. {sample:30s} (數字個數: {operand_count})")

print("\n【評估集範例（兩個數字）】")
print("-" * 60)
for i in range(min(10, len(df_eval_2ops))):
    sample = df_eval_2ops.iloc[i]['src']
    operand_count = count_operands(sample)
    print(f"{i+1:2d}. {sample:30s} (數字個數: {operand_count})")

print("=" * 60)
print()

篩選數據集

正在篩選訓練集（三個數字的運算）...
✓ 訓練集篩選完成: 2,362,466 筆（三個數字）

正在篩選評估集（兩個數字的運算）...
✓ 評估集篩選完成: 716 筆（兩個數字）

【訓練集範例（三個數字）】
------------------------------------------------------------
 1. 14*(43+20)=882                 (數字個數: 3)
 2. (6+1)*5=35                     (數字個數: 3)
 3. 13+32+29=74                    (數字個數: 3)
 4. 31*(3-11)=-248                 (數字個數: 3)
 5. 24*49+1=1177                   (數字個數: 3)
 6. 3+(25*25)=628                  (數字個數: 3)
 7. 8*(30+10)=320                  (數字個數: 3)
 8. 9*38+49=391                    (數字個數: 3)
 9. 23-26*15=-367                  (數字個數: 3)
10. 18*22-19=377                   (數字個數: 3)

【評估集範例（兩個數字）】
------------------------------------------------------------
 1. 41*33=1353                     (數字個數: 2)
 2. 12-28=-16                      (數字個數: 2)
 3. 17*46=782                      (數字個數: 2)
 4. 25*16=400                      (數字個數: 2)
 5. 8+30=38                        (數字個數: 2)
 6. 43*3=129                       (數字個數: 2)
 7. 21*38=798               

## 步驟 4: 處理篩選後的數據

In [48]:
print("=" * 60)
print("處理篩選後的訓練集和評估集")
print("=" * 60)

# 處理訓練集（三個數字）
print("\n正在處理訓練集（三個數字的運算）...")
start_time = time.time()
df_train_3ops_processed = process_data(df_train_3ops)
end_time = time.time()
print(f"✓ 訓練集處理完成，耗時: {end_time - start_time:.2f}秒")

# 處理評估集（兩個數字）
print("\n正在處理評估集（兩個數字的運算）...")
start_time = time.time()
df_eval_2ops_processed = process_data(df_eval_2ops)
end_time = time.time()
print(f"✓ 評估集處理完成，耗時: {end_time - start_time:.2f}秒")

# 創建 Dataset 和 DataLoader
ds_train_3ops = Dataset(df_train_3ops_processed[['char_id_list', 'label_id_list']])
ds_eval_2ops = Dataset(df_eval_2ops_processed[['char_id_list', 'label_id_list']])

dl_train_3ops = torch.utils.data.DataLoader(
    ds_train_3ops,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=4
)

dl_eval_2ops = torch.utils.data.DataLoader(
    ds_eval_2ops,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=4
)

print(f"\n✓ DataLoader 已創建")
print(f"  訓練集批次數: {len(dl_train_3ops):,}")
print(f"  評估集批次數: {len(dl_eval_2ops):,}")
print("=" * 60)
print()

處理篩選後的訓練集和評估集

正在處理訓練集（三個數字的運算）...
Processing 2362466 samples...


100%|██████████| 2362466/2362466 [01:33<00:00, 25205.76it/s] 


✓ 訓練集處理完成，耗時: 94.58秒

正在處理評估集（兩個數字的運算）...
Processing 716 samples...


100%|██████████| 716/716 [00:00<00:00, 445104.74it/s]

✓ 評估集處理完成，耗時: 0.01秒

✓ DataLoader 已創建
  訓練集批次數: 770
  評估集批次數: 1



## 步驟 5: 重置訓練參數並創建新模型

In [49]:
# 清空 GPU 緩存
torch.cuda.empty_cache()

# 重新建立模型（使用不同的隨機種子）
torch.manual_seed(123)  # 使用不同的種子
model_3to2 = CharLSTM(vocab_size, embed_dim, hidden_dim)
print("LSTM (三個數字訓練 → 兩個數字驗證)")
model_3to2 = model_3to2.to(device)

# 編譯模型
model_3to2 = torch.compile(model_3to2, dynamic=True)

# 重新建立損失函數和優化器
criterion_3to2 = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
optimizer_3to2 = torch.optim.AdamW(model_3to2.parameters(), lr=lr, weight_decay=1e-4)
scheduler_3to2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_3to2, T_max=epochs, eta_min=lr * 0.1)

print("=" * 60)
print("✓ 三個數字 → 兩個數字實驗模型已準備就緒")
print(f"模型參數量: {sum(p.numel() for p in model_3to2.parameters()):,}")
print(f"初始學習率: {lr}")
print(f"訓練輪數: {epochs}")
print(f"訓練集: {len(df_train_3ops):,} 筆（三個數字）")
print(f"評估集: {len(df_eval_2ops):,} 筆（兩個數字）")
print("=" * 60)
print()

LSTM (三個數字訓練 → 兩個數字驗證)
✓ 三個數字 → 兩個數字實驗模型已準備就緒
模型參數量: 1,128,210
初始學習率: 0.001
訓練輪數: 30
訓練集: 2,362,466 筆（三個數字）
評估集: 716 筆（兩個數字）



## 步驟 6: 訓練模型（三個數字訓練集 → 兩個數字評估集）

In [50]:
from copy import deepcopy

model_3to2 = model_3to2.to(device)
model_3to2.train()
i = 0

# 記錄最佳模型
best_em_3to2 = 0
best_model_state_3to2 = None

import time
from datetime import datetime, timedelta

# 記錄訓練時間
training_start_time = time.time()
start_datetime = datetime.now()

print("=" * 60)
print(f"【三個數字→兩個數字實驗】訓練開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print()

for epoch in range(1, epochs+1):
    print(f"Epoch {epoch} start at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    bar = tqdm(dl_train_3ops, desc=f"Train (3 operands) epoch {epoch}")
    
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        optimizer_3to2.zero_grad()
        
        batch_pred_y = model_3to2(batch_x.to(device), batch_x_lens)
        loss = criterion_3to2(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model_3to2.parameters(), grad_clip)
        optimizer_3to2.step()
        
        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss=loss.item())
    
    # ============================================
    # 評估：在兩個數字的評估集上測試
    # ============================================
    model_3to2.eval()
    bar = tqdm(dl_eval_2ops, desc=f"Validation (2 operands) epoch {epoch}")
    matched = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
            batch_pred_y = model_3to2(batch_x.to(device), batch_x_lens)
            
            for idx in range(len(batch_x)):
                x_seq = batch_x[idx][:batch_x_lens[idx]].tolist()
                x_str = ''.join([id_to_char[id] for id in x_seq])
                
                equal_pos = x_str.find('=')
                if equal_pos == -1:
                    continue
                
                pred_logits = batch_pred_y[idx][equal_pos:batch_x_lens[idx]-1]
                pred_ids = pred_logits.argmax(dim=-1).cpu().tolist()
                
                pred_chars = []
                for pred_id in pred_ids:
                    if pred_id == char_to_id['<eos>']:
                        break
                    pred_chars.append(id_to_char[pred_id])
                pred_answer = ''.join(pred_chars)
                
                ground_truth_answer = x_str[equal_pos+1:]
                
                if pred_answer == ground_truth_answer:
                    matched += 1
                total += 1
            
            bar.set_postfix(EM=matched/total if total > 0 else 0)
    
    current_em = matched/total if total > 0 else 0
    print(f"\nEpoch {epoch} - Exact Match (兩個數字評估集): {current_em:.4f} ({matched}/{total})")
    
    # 保存最佳模型
    if current_em > best_em_3to2:
        best_em_3to2 = current_em
        best_model_state_3to2 = deepcopy(model_3to2.state_dict())
        print(f"✓ 新的最佳模型！EM = {best_em_3to2:.4f}")
    
    # 展示生成例子（測試兩個數字的運算）
    print("\n=== 生成例子（兩個數字運算）===")
    test_cases = ['1+1=', '10+20=', '5*6=', '44-3=', '99-50=', '7*8=', '15+25=', '100-1=']
    for test in test_cases:
        result = model_3to2.generator(test, max_len=50)
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        
        try:
            correct_answer = str(eval(test[:-1]))
        except:
            correct_answer = "N/A"
        
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} {test:15s} | 生成: {answer:10s} | 正確: {correct_answer}")
    
    # 展示三個數字運算的例子（測試訓練集類型）
    print("\n=== 生成例子（三個數字運算 - 訓練集類型）===")
    test_cases_3 = ['1+2+3=', '10*5+3=', '20-5-3=', '2*3*4=']
    for test in test_cases_3:
        result = model_3to2.generator(test, max_len=50)
        equal_pos = test.find('=')
        if '<eos>' in result:
            eos_pos = result.find('<eos>')
            answer = result[equal_pos+1:eos_pos]
        else:
            answer = result[equal_pos+1:]
        
        try:
            correct_answer = str(eval(test[:-1]))
        except:
            correct_answer = "N/A"
        
        match = "✓" if answer == correct_answer else "✗"
        print(f"{match} {test:15s} | 生成: {answer:10s} | 正確: {correct_answer}")
    
    # 更新學習率
    scheduler_3to2.step()
    current_lr = optimizer_3to2.param_groups[0]['lr']
    print(f"當前學習率: {current_lr:.6f}")
    print()
    
    model_3to2.train()

# ============================================
# 訓練結束
# ============================================
training_end_time = time.time()
end_datetime = datetime.now()
training_duration = training_end_time - training_start_time

print("=" * 60)
print("【三個數字→兩個數字實驗】訓練完成！")
print("=" * 60)
print(f"開始時間: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"結束時間: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"訓練持續時間: {str(timedelta(seconds=int(training_duration)))}")
print(f"總計: {training_duration:.2f} 秒 ({training_duration/60:.2f} 分鐘)")
print("-" * 60)
print(f"最佳 Exact Match (兩個數字評估集): {best_em_3to2:.4f}")
print("=" * 60)
print()

# 創建保存目錄
import os
save_dir = './checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 保存模型
if best_model_state_3to2 is not None:
    model_3to2.load_state_dict(best_model_state_3to2)
    model_3to2_path = os.path.join(save_dir, 'best_model_3ops_to_2ops.pth')
    torch.save({
        'model_state_dict': best_model_state_3to2,
        'vocab_size': vocab_size,
        'embed_dim': embed_dim,
        'hidden_dim': hidden_dim,
        'char_to_id': char_to_id,
        'id_to_char': id_to_char,
        'best_em': best_em_3to2,
        'train_operands': 3,
        'eval_operands': 2,
    }, model_3to2_path)
    print(f"✓ 三個數字→兩個數字實驗模型已保存至: {model_3to2_path}")
    print("=" * 60)

【三個數字→兩個數字實驗】訓練開始時間: 2025-10-05 13:29:39

Epoch 1 start at 2025-10-05 13:29:39


Validation (2 operands) epoch 1: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it, EM=0.0196]



Epoch 1 - Exact Match (兩個數字評估集): 0.0196 (14/716)
✓ 新的最佳模型！EM = 0.0196

=== 生成例子（兩個數字運算）===
✗ 1+1=            | 生成: 14         | 正確: 2
✗ 10+20=          | 生成: 40         | 正確: 30
✗ 5*6=            | 生成: -35        | 正確: 30
✗ 44-3=           | 生成: 63         | 正確: 41
✗ 99-50=          | 生成: 24         | 正確: 49
✗ 7*8=            | 生成: -38        | 正確: 56
✗ 15+25=          | 生成: 48         | 正確: 40
✗ 100-1=          | 生成: 27         | 正確: 99

=== 生成例子（三個數字運算 - 訓練集類型）===
✗ 1+2+3=          | 生成: 8          | 正確: 6
✗ 10*5+3=         | 生成: 67         | 正確: 53
✗ 20-5-3=         | 生成: 14         | 正確: 12
✗ 2*3*4=          | 生成: 22         | 正確: 24
當前學習率: 0.000998

Epoch 2 start at 2025-10-05 13:29:57


Validation (2 operands) epoch 2: 100%|██████████| 1/1 [00:02<00:00,  2.67s/it, EM=0.0168]



Epoch 2 - Exact Match (兩個數字評估集): 0.0168 (12/716)

=== 生成例子（兩個數字運算）===
✗ 1+1=            | 生成: 12         | 正確: 2
✗ 10+20=          | 生成: 40         | 正確: 30
✗ 5*6=            | 生成: 170        | 正確: 30
✗ 44-3=           | 生成: 65         | 正確: 41
✗ 99-50=          | 生成: 22         | 正確: 49
✗ 7*8=            | 生成: -28        | 正確: 56
✗ 15+25=          | 生成: 50         | 正確: 40
✗ 100-1=          | 生成: 27         | 正確: 99

=== 生成例子（三個數字運算 - 訓練集類型）===
✗ 1+2+3=          | 生成: 8          | 正確: 6
✗ 10*5+3=         | 生成: 63         | 正確: 53
✓ 20-5-3=         | 生成: 12         | 正確: 12
✗ 2*3*4=          | 生成: 22         | 正確: 24
當前學習率: 0.000990

Epoch 3 start at 2025-10-05 13:30:15


...


Epoch 28 - Exact Match (兩個數字評估集): 0.0168 (12/716)

=== 生成例子（兩個數字運算）===
✗ 1+1=            | 生成: 10         | 正確: 2
✗ 10+20=          | 生成: 40         | 正確: 30
✗ 5*6=            | 生成: -20        | 正確: 30
✗ 44-3=           | 生成: 65         | 正確: 41
✗ 99-50=          | 生成: 21         | 正確: 49
✗ 7*8=            | 生成: -48        | 正確: 56
✗ 15+25=          | 生成: 55         | 正確: 40
✗ 100-1=          | 生成: 21         | 正確: 99

=== 生成例子（三個數字運算 - 訓練集類型）===
✓ 1+2+3=          | 生成: 6          | 正確: 6
✓ 10*5+3=         | 生成: 53         | 正確: 53
✓ 20-5-3=         | 生成: 12         | 正確: 12
✓ 2*3*4=          | 生成: 24         | 正確: 24
當前學習率: 0.000110

Epoch 29 start at 2025-10-05 13:37:58


Validation (2 operands) epoch 29: 100%|██████████| 1/1 [00:02<00:00,  2.39s/it, EM=0.0154]



Epoch 29 - Exact Match (兩個數字評估集): 0.0154 (11/716)

=== 生成例子（兩個數字運算）===
✗ 1+1=            | 生成: 10         | 正確: 2
✗ 10+20=          | 生成: 40         | 正確: 30
✗ 5*6=            | 生成: 26         | 正確: 30
✗ 44-3=           | 生成: 65         | 正確: 41
✗ 99-50=          | 生成: 21         | 正確: 49
✗ 7*8=            | 生成: -48        | 正確: 56
✗ 15+25=          | 生成: 55         | 正確: 40
✗ 100-1=          | 生成: 21         | 正確: 99

=== 生成例子（三個數字運算 - 訓練集類型）===
✓ 1+2+3=          | 生成: 6          | 正確: 6
✓ 10*5+3=         | 生成: 53         | 正確: 53
✓ 20-5-3=         | 生成: 12         | 正確: 12
✓ 2*3*4=          | 生成: 24         | 正確: 24
當前學習率: 0.000102

Epoch 30 start at 2025-10-05 13:38:16


Validation (2 operands) epoch 30: 100%|██████████| 1/1 [00:02<00:00,  2.40s/it, EM=0.0154]


Epoch 30 - Exact Match (兩個數字評估集): 0.0154 (11/716)

=== 生成例子（兩個數字運算）===
✗ 1+1=            | 生成: 10         | 正確: 2
✗ 10+20=          | 生成: 40         | 正確: 30
✗ 5*6=            | 生成: -20        | 正確: 30
✗ 44-3=           | 生成: 65         | 正確: 41
✗ 99-50=          | 生成: 21         | 正確: 49
✗ 7*8=            | 生成: -48        | 正確: 56
✗ 15+25=          | 生成: 55         | 正確: 40
✗ 100-1=          | 生成: 21         | 正確: 99

=== 生成例子（三個數字運算 - 訓練集類型）===
✓ 1+2+3=          | 生成: 6          | 正確: 6
✓ 10*5+3=         | 生成: 53         | 正確: 53
✓ 20-5-3=         | 生成: 12         | 正確: 12
✓ 2*3*4=          | 生成: 24         | 正確: 24
當前學習率: 0.000100

【三個數字→兩個數字實驗】訓練完成！
開始時間: 2025-10-05 13:29:39
結束時間: 2025-10-05 13:38:34
訓練持續時間: 0:08:55
總計: 535.35 秒 (8.92 分鐘)
------------------------------------------------------------
最佳 Exact Match (兩個數字評估集): 0.0196

✓ 三個數字→兩個數字實驗模型已保存至: ./checkpoints/best_model_3ops_to_2ops.pth
